# EDGAR to fluxy 

Example how to convert EDGAR to fluxy format.

Edgar processing taken from the edgar tutorial.


## EDGAR Files

In this section, we will examine the EDGAR NetCDF files.

These files can be downloaded directly from the [EDGAR website](https://edgar.jrc.ec.europa.eu/emissions_data_and_maps). However, the most convenient method is to use the `download_edgar_files` function provided by emiproc, as demonstrated below.


In [ ]:
from pathlib import Path
from emiproc.inventories.edgar import download_edgar_files
from emiproc import FILES_DIR
import matplotlib.pyplot as plt


plt.style.use("default")

local_dir = FILES_DIR / "edgar"
local_dir.mkdir(exist_ok=True)

years = [2020, 2021, 2022]

In [ ]:
for year in years:
    download_edgar_files(local_dir / "data"/  str(year), year=year, substances=["CH4", "CO2", "CO2bio"])


Creating code to parse inventory files for every project can be cumbersome.

The goal of emiproc is to provide a straightforward interface for reading emissions inventories. In this case, you can use the `EDGARv8` class, which reads the EDGAR files and loads them as an `Inventory` object.


In [ ]:
from emiproc.inventories.edgar import EDGARv8

# Load the edgar inventory

invs = [EDGARv8(local_dir / "data"/  str(year) / "v8.0_*.nc", year=year) for year in years]
invs

### Remapping the Inventory

In this section, we will remap the inventory to a grid somehwere over europe and also until the east coast of the US.



In [ ]:
from emiproc.grids import RegularGrid

grid = RegularGrid(xmin=-74, xmax=7, ymin=35.5, ymax=71.2, nx=170, ny=120)
grid

In some cases, you may already have a grid for your simulation, which you could import into emiproc with appropriate code.
A [tutorial for developer](
    https://emiproc.readthedocs.io/en/master/contrib/create_grid.html
)
 is available. 

In [ ]:
# Remap the inventory on the grid
from emiproc.regrid import remap_inventory


remappeds = [remap_inventory(inv, grid, weights_file=local_dir / ".weights_remap_fluxygrid_2") for inv in invs]
remappeds

Let's examine our remapped inventory.

In [ ]:
from emiproc.plots import plot_inventory


plot_inventory(remappeds[0], total_only=True)

For this domain, the panorama is quite different.
For example, waste is well-managed in Australia, so its fraction of the total emissions is smaller.
Maybe Australia exports its waste to other regions such as Asia or Africa ? Who knows ?

### Grouping Categories

Many categories are similar and have very small emissions.

In such cases, we can group some categories together to reduce the number of emission sectors.

In [ ]:
from emiproc.inventories.utils import group_categories
from emiproc.speciation import merge_substances

group = lambda inv : group_categories(
    merge_substances(inv,{"CO2": ["CO2", "CO2bio"]}),
    categories_group={
        "agriculture": [
            "Agricultural soils",
            "Agricultural waste burning",
            "Manure management",
        ],
        "industry": [
            "Chemical processes",
            "Power Industry",
            "Oil refineries and Transformation industry",
            "Fuel exploitation",
            "Energy for buildings",
            "Combustion for manufacturing",
            "Iron and steel production",
            "Non energy use of fuels",
            "Solvents and products use",
            "Non-ferrous metals production",
            "Non-metallic minerals production",
        ],
        "livestock": ["Enteric fermentation"],
        "waste": [
            "Waste water handling",
            "Solid waste incineration",
            "Solid waste landfills",
        ],
        "transportation": [
            "Aviation climbing_and_descent",
            "Aviation cruise",
            "Aviation landing_and_takeoff",
            "Railways, pipelines, off-road transport",
            "Shipping",
            "Road transportation",
        ],
    },
)
grouppeds = [group(inv) for inv in remappeds]
grouppeds[0].categories

## Exporting the Inventory

One of the main features of emiproc is its ability to export the inventory to various types of simulation inputs.

In this example, we will save the inventory to a NetCDF file.


In [ ]:
from emiproc.profiles.temporal.io import from_yaml

yml_dir = FILES_DIR / "profiles" / "yamls"
profiles = {
    "transportation": from_yaml(yml_dir / "light_daily.yaml"),
    "industry": from_yaml(yml_dir / "industry_daily.yaml"),
    "agriculture": from_yaml(yml_dir / "no_factor.yaml"),
    "livestock": from_yaml(yml_dir / "no_factor.yaml"),
    "waste": from_yaml(yml_dir / "no_factor.yaml"),
}

for inv in grouppeds:
    for category, profile in profiles.items():
        inv.set_profile(profile, category=category)

In [ ]:
from emiproc.exports.fluxy import export_fluxy

# Export the inventory to fluxy format
export_fluxy(grouppeds, local_dir / "fluxy" / "edgar_emiproc", transport_model="EmiprocEdgar")

## Using fluxy to plot the inventory

First we need to read the data, this is done wit the `read_model_output` function.

In [ ]:
from fluxy.io import  read_model_output

dss_fluxy = read_model_output(
    
    data_dir=local_dir / "fluxy" / "edgar_emiproc",
    file_type='flux', # Fluxy can plot also concentrations, but in emiproc we only use fluxes
    species="CH4",
    models=["EmiprocEdgar"],
)
dss_fluxy

In [ ]:
from fluxy.plots.flux_map import plot_flux_map


# This function is a simple flux map
fig = plot_flux_map(
    dss_fluxy,
    species='CO2',
    # Set this to only see the inventory
    # In fluxy they run inversion results, in our case we only have the prior inventory
    only = "prior",
)

In [ ]:
from fluxy.plots.flux_map import plot_flux_map_over_time


# This function plots the map over time (years)
fig = plot_flux_map_over_time(
    dss_fluxy,
    var="flux_total_prior",
    species="CO2",
)

In [ ]:
from fluxy.plots.flux_timeseries import  plot_country_flux

# This function plots timeseries of the total fluxes for each country
plot_country_flux(
    dss_fluxy,
    species='CO2', 
    # Set to false, otherwise it will try to look for files you don't have
    plot_inventory=False, 
)